# Nadun Chandrabahu- EM/F1 score calculation using own data

I'm using MCTest dataset (Richardson et al, 2013), source: https://huggingface.co/datasets/sagnikrayc/mctest

In [32]:
%%capture
%pip install datasets
%pip install transformers
%pip install scikit-learn
%pip install torch
#install if necessary, i'm using Python 3.12.7

In [ ]:
from datasets import load_dataset
import pandas as pd

ds = load_dataset("sagnikrayc/mctest", "mc500")
ds=ds["test"]

In [30]:
df=pd.DataFrame(ds)
df['answer_text'] = df.apply(lambda row: row['answer_options'].get(row['answer'], None), axis=1)
mctest_df = df[['question', 'story', 'answer_text']]
mctest_df.head()

,question,story,answer_text
0,What time did the party start?,It was Sally's birthday. She was very excited....,1
1,Who got hurt at the party?,It was Sally's birthday. She was very excited....,Jennifer and Sally
2,Whose birthday is it?,It was Sally's birthday. She was very excited....,Sally
3,What time did Jennifer arrive to the party?,It was Sally's birthday. She was very excited....,2
4,What did the piggies do when Andy got back fro...,On the farm there was a little piggy named And...,play games and eat dinner


In [33]:
#importing model
from transformers import pipeline
import torch
distilBert=pipeline("question-answering",model="distilbert-base-uncased-distilled-squad")

In [36]:
#make predictions using distilbert, takes about 2 minutes to run
mctest_df["model_prediction"]=mctest_df.apply(lambda row: distilBert(question=row["question"],context=row["story"]),axis=1)
mctest_df['predicted_answer'] = mctest_df['model_prediction'].apply(lambda x: x.get('answer', None))

C:\Users\nadun\AppData\Local\Temp\ipykernel_16976\3080497688.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mctest_df["model_prediction"]=mctest_df.apply(lambda row: distilBert(question=row["question"],context=row["story"]),axis=1)
C:\Users\nadun\AppData\Local\Temp\ipykernel_16976\3080497688.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mctest_df['predicted_answer'] = mctest_df['model_prediction'].apply(lambda x: x.get('answer', None))


In [38]:
mctest_df.head()
mctest_df['score'] = mctest_df['model_prediction'].apply(lambda x: x.get('score', None))
mctest_df['exact_match'] = (mctest_df['answer_text'] == mctest_df['predicted_answer']).astype(int)

C:\Users\nadun\AppData\Local\Temp\ipykernel_16976\1482272627.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mctest_df['score'] = mctest_df['model_prediction'].apply(lambda x: x.get('score', None))
C:\Users\nadun\AppData\Local\Temp\ipykernel_16976\1482272627.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mctest_df['exact_match'] = (mctest_df['answer_text'] == mctest_df['predicted_answer']).astype(int)


In [ ]:
#calculate EM score 
average_score = mctest_df[mctest_df['exact_match'] == 1]['score'].mean()
number_of_exact_matches=mctest_df[mctest_df['exact_match'] == 1]['score'].count()
print("Average Score of Exact Matches is:",average_score*100, "on", number_of_exact_matches, "Exact Matches.")

Average Score of Exact Matches is: 76.17681198910381 on 89 Exact Matches.


In [46]:
# calculate F1 Score
# calculate F1 score
from sklearn.metrics import f1_score

def calculate_f1(row):
    true_answer = str(row["answer_text"]).split()
    predicted_answer = str(row["predicted_answer"]).split()

    # Calculate precision and recall
    common = set(true_answer) & set(predicted_answer)
    precision = len(common) / len(predicted_answer) if predicted_answer else 0
    recall = len(common) / len(true_answer) if true_answer else 0

    # Calculate F1 score
    if precision + recall == 0:
        f1 = 0
    else:
        f1 = 2 * (precision * recall) / (precision + recall)
    
    return f1

# Apply the function to each row and store the F1 scores in a new column
mctest_df['f1_score'] = mctest_df.apply(calculate_f1, axis=1)
print("F1 Score:",mctest_df["f1_score"].mean()*100)


F1 Score: 31.22289366167542
